In [23]:
from imblearn.under_sampling import NearMiss
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import TomekLinks
from imblearn.under_sampling import RandomUnderSampler  
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import TomekLinks
from imblearn.over_sampling import ADASYN
from sklearn.metrics import *
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats import weightstats
from statsmodels.stats import proportion
from scipy.stats import t
import warnings 
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [49]:
df = pd.read_csv('../dataset/BankChurners.csv').iloc[:,1:-2]
df.shape

(10127, 20)

## undersampling

In [50]:
X = df.drop(columns='Attrition_Flag')
y = df['Attrition_Flag']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 123)

In [51]:
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(7088, 19) (3039, 19)
(7088,) (3039,)


In [53]:
under_sampler = RandomUnderSampler(random_state = 123)
under_train_x, under_train_y = under_sampler.fit_resample(X_train, y_train)

under_train_x = pd.DataFrame(under_train_x, columns = X.columns)
under_train_y = pd.Series(under_train_y)

print(under_train_x.shape)
print(under_train_y.shape)
print('------------------')
print(under_train_y.value_counts())

(2320, 19)
(2320,)
------------------
Existing Customer    1160
Attrited Customer    1160
Name: Attrition_Flag, dtype: int64


In [54]:
df = pd.concat([under_train_x, under_train_y], axis = 1)

## 1. 신용카드 한도 차이
* 귀무가설 : 이탈고객과 기존고객 사이의 신용한도는 차이가 없다.
-> 이탈고객의 신용한도 평균과 기존고객 신용한도 평균의 차이는 0이다.
-> Existing_mean - Attrited_mean = 0
* 대립가설 : 이탈고객과 기존고객 사이의 신용한도는 차이가 있다.
-> 이탈고객의 신용한도 평균과 기존고객 신용한도 평균의 차이는 0이 아니다.
-> Existing_mean - Attrited_mean != 0


등분산검정(F-test)
귀무가설 : 두 집단의 분산은 차이 없다.
대립가설 : 두 집단의 분산은 차이가 있다.
-> pvalue값이 0.05보다 작으면 귀무가설을 기각하니, 이분산/ 크면 귀무가설 기각을 못하니까 등분산.




In [56]:
from scipy import stats
Credit_Limit = df.Credit_Limit 

#독립표본 t-검정
#변수생성
Existing = df[df.Attrition_Flag == "Existing Customer"] #Existing Customer
Attrited = df[df.Attrition_Flag == "Attrited Customer"] #Attrited Customer

#Levene의 등분산 검정 
lresult = stats.levene(Existing.Credit_Limit, Attrited.Credit_Limit)
print('LeveneResult(F) : %.3f \np-value : %.3f' % (lresult))
if lresult.pvalue >= 0.05:
    bool_ = True
    print("-> 신용카드 한도 평균의 분산은 이분산이다.")
else:
    bool_ = False
    print('-> 신용카드 한도 평균의 분산은 등분산이다.')

result = stats.ttest_ind(Existing.Credit_Limit, Attrited.Credit_Limit, equal_var=bool_) 
print('t statistic : %.3f \np-value : %.3f' % (result))
if result.pvalue >= 0.05:
    print("--> 이탈고객의 신용카드 한도 평균과 기존고객 신용카드 한도 평균의 차이는 0이므로 귀무가설을 채택한다.")
else:
    print('--> 이탈고객의 신용카드 한도 평균과 기존고객 신용카드 한도 평균의 차이는 0이 아니므로 귀무가설을 기각한다.')

LeveneResult(F) : 0.259 
p-value : 0.611
-> 신용카드 한도 평균의 분산은 이분산이다.
t statistic : 0.374 
p-value : 0.708
--> 이탈고객의 신용카드 한도 평균과 기존고객 신용카드 한도 평균의 차이는 0이므로 귀무가설을 채택한다.


## 2. 신용카드 총 회전잔액 차이 
* 귀무가설 : 이탈고객과 기존고객 사이의 신용카드 총 회전잔액 평균의 차이가 없다.
-> 이탈고객의 신용한도 평균과 기존고객 신용카드 총 회전잔액 평균의 차이는 0이다.
-> Existing_mean - Attrited_mean = 0
* 대립가설 : 이탈고객과 기존고객 사이의 신용카드 총 회전잔액 평균의 차이가 있다.
-> 이탈고객의 신용한도 평균과 기존고객 신용카드 총 회전잔액 평균의 차이는 0이 아니다.
-> Existing_mean - Attrited_mean != 0

In [57]:
from scipy import stats
Total_Revolving_Bal = df.Total_Revolving_Bal

#독립표본 t-검정
#변수생성
Existing = df[df.Attrition_Flag == "Existing Customer"] #Existing Customer
Attrited = df[df.Attrition_Flag == "Attrited Customer"] #Attrited Customer

#Levene의 등분산 검정 
lresult = stats.levene(Existing.Total_Revolving_Bal, Attrited.Total_Revolving_Bal)
print('LeveneResult(F) : %.3f \np-value : %.3f' % (lresult))
if lresult.pvalue >= 0.05:
    bool_ = True
    print("-> 신용카드 총 회전잔액 평균의 분산은 이분산이다.")
else:
    bool_ = False
    print('-> 신용카드 총 회전잔액 평균의 분산은 등분산이다.')

result = stats.ttest_ind(Existing.Total_Revolving_Bal, Attrited.Total_Revolving_Bal, equal_var=bool_) 
print('t statistic : %.3f \np-value : %.3f' % (result))
if result.pvalue >= 0.05:
    print("--> 이탈고객의 신용카드 총 회전잔액 평균과 기존고객 신용카드 총 회전잔액 평균의 차이는 0이므로 귀무가설을 채택한다.")
else:
    print('--> 이탈고객의 신용카드 총 회전잔액 평균과 기존고객 신용카드 총 회전잔액 평균의 차이는 0이 아니므로 귀무가설을 기각한다.')

LeveneResult(F) : 4.883 
p-value : 0.027
-> 신용카드 총 회전잔액 평균의 분산은 등분산이다.
t statistic : 17.400 
p-value : 0.000
--> 이탈고객의 신용카드 총 회전잔액 평균과 기존고객 신용카드 총 회전잔액 평균의 차이는 0이 아니므로 귀무가설을 기각한다.


## 3. 신용 한도 구매 가능 차이 
* 귀무가설 : 이탈고객과 기존고객 사이의 신용 한도 구매 가능 평균의 차이가 없다.
-> 이탈고객의 신용 한도 구매 가능 평균과 기존고객 신용 한도 구매 가능 평균의 차이는 0이다.
-> Existing_mean - Attrited_mean = 0
* 대립가설 : 이탈고객과 기존고객 사이의 신용 한도 구매 가능 평균의 차이가 있다.
-> 이탈고객의 신용 한도 구매 가능 평균과 기존고객 신용 한도 구매 가능 평균의 차이는 0이 아니다.
-> Existing_mean - Attrited_mean != 0

In [58]:
from scipy import stats
Avg_Open_To_Buy  = df.Avg_Open_To_Buy 

#독립표본 t-검정
#변수생성
Existing = df[df.Attrition_Flag == "Existing Customer"] #Existing Customer
Attrited = df[df.Attrition_Flag == "Attrited Customer"] #Attrited Customer

#Levene의 등분산 검정 
lresult = stats.levene(Existing.Avg_Open_To_Buy , Attrited.Avg_Open_To_Buy )
print('LeveneResult(F) : %.3f \np-value : %.3f' % (lresult))
if lresult.pvalue >= 0.05:
    bool_ = True
    print("-> 신용 한도 구매 가능 평균의 분산은 이분산이다.")
else:
    bool_ = False
    print('-> 신용 한도 구매 가능 평균의 분산은 등분산이다.')

result = stats.ttest_ind(Existing.Avg_Open_To_Buy , Attrited.Avg_Open_To_Buy , equal_var=bool_) 
print('t statistic : %.3f \np-value : %.3f' % (result))
if result.pvalue >= 0.05:
    print("--> 이탈고객의 신용 한도 구매 가능 평균과 기존고객 신용 한도 구매 가능 평균의 차이는 0이므로 귀무가설을 채택한다.")
else:
    print('--> 이탈고객의 신용 한도 구매 가능 평균과 기존고객 신용 한도 구매 가능 평균의 차이는 0이 아니므로 귀무가설을 기각한다.')

LeveneResult(F) : 0.268 
p-value : 0.604
-> 신용 한도 구매 가능 평균의 분산은 이분산이다.
t statistic : -1.244 
p-value : 0.213
--> 이탈고객의 신용 한도 구매 가능 평균과 기존고객 신용 한도 구매 가능 평균의 차이는 0이므로 귀무가설을 채택한다.


## 4. 거래 금액 변경  
* 귀무가설 : 이탈고객과 기존고객 사이의 거래 금액 변경 평균의 차이가 없다.
-> 이탈고객의 거래 금액 변경 평균과 기존고객 거래 금액 변경 평균의 차이는 0이다.
-> Existing_mean - Attrited_mean = 0
* 대립가설 : 이탈고객과 기존고객 사이의 거래 금액 변경 평균의 차이가 있다.
-> 이탈고객의 거래 금액 변경 평균과 기존고객 거래 금액 변경 평균의 차이는 0이 아니다.
-> Existing_mean - Attrited_mean != 0

In [59]:
from scipy import stats
Total_Amt_Chng_Q4_Q1   = df.Total_Amt_Chng_Q4_Q1  

#독립표본 t-검정
#변수생성
Existing = df[df.Attrition_Flag == "Existing Customer"] #Existing Customer
Attrited = df[df.Attrition_Flag == "Attrited Customer"] #Attrited Customer

#Levene의 등분산 검정 
lresult = stats.levene(Existing.Total_Amt_Chng_Q4_Q1  , Attrited.Total_Amt_Chng_Q4_Q1  )
print('LeveneResult(F) : %.3f \np-value : %.3f' % (lresult))
if lresult.pvalue >= 0.05:
    bool_ = True
    print("-> 거래 금액 변경 차이 평균의 분산은 이분산이다.")
else:
    bool_ = False
    print('-> 거래 금액 변경 차이 평균의 분산은 등분산이다.')

result = stats.ttest_ind(Existing.Total_Amt_Chng_Q4_Q1  , Attrited.Total_Amt_Chng_Q4_Q1  , equal_var=bool_) 
print('t statistic : %.3f \np-value : %.3f' % (result))
if result.pvalue >= 0.05:
    print("--> 이탈고객의 거래 금액 변경 차이 평균과 기존고객 거래 금액 변경 차이 평균의 차이는 0이므로 귀무가설을 채택한다.")
else:
    print('--> 이탈고객의 거래 금액 변경 차이 평균과 기존고객 거래 금액 변경 차이 평균의 차이는 0이 아니므로 귀무가설을 기각한다.')

LeveneResult(F) : 7.378 
p-value : 0.007
-> 거래 금액 변경 차이 평균의 분산은 등분산이다.
t statistic : 8.794 
p-value : 0.000
--> 이탈고객의 거래 금액 변경 차이 평균과 기존고객 거래 금액 변경 차이 평균의 차이는 0이 아니므로 귀무가설을 기각한다.


## 5. 총 거래 금액
* 귀무가설 : 이탈고객과 기존고객 사이의 총 거래 금액 평균의 차이가 없다.
-> 이탈고객의 총 거래 금액 평균과 기존고객 총 거래 금액 평균의 차이는 0이다.
-> Existing_mean - Attrited_mean = 0
* 대립가설 : 이탈고객과 기존고객 사이의 총 거래 금액 평균의 차이가 있다.
-> 이탈고객의 총 거래 금액 평균과 기존고객 총 거래 금액 평균의 차이는 0이 아니다.
-> Existing_mean - Attrited_mean != 0

In [60]:
from scipy import stats
Total_Trans_Amt    = df.Total_Trans_Amt   

#독립표본 t-검정
#변수생성
Existing = df[df.Attrition_Flag == "Existing Customer"] #Existing Customer
Attrited = df[df.Attrition_Flag == "Attrited Customer"] #Attrited Customer

#Levene의 등분산 검정 
lresult = stats.levene(Existing.Total_Trans_Amt, Attrited.Total_Trans_Amt)
print('LeveneResult(F) : %.3f \np-value : %.3f' % (lresult))
if lresult.pvalue >= 0.05:
    bool_ = True
    print("-> 총 거래 금액 평균의 분산은 이분산이다.")
else:
    bool_ = False
    print('-> 총 거래 금액 평균의 분산은 등분산이다.')

result = stats.ttest_ind(Existing.Total_Trans_Amt, Attrited.Total_Trans_Amt, equal_var=bool_) 
print('t statistic : %.3f \np-value : %.3f' % (result))
if result.pvalue >= 0.05:
    print("--> 이탈고객의 총 거래 금액 평균과 기존고객 총 거래 금액 평균의 차이는 0이므로 귀무가설을 채택한다.")
else:
    print('--> 이탈고객의 총 거래 금액 평균과 기존고객 총 거래 금액 평균의 차이는 0이 아니므로 귀무가설을 기각한다.')

LeveneResult(F) : 37.889 
p-value : 0.000
-> 총 거래 금액 평균의 분산은 등분산이다.
t statistic : 10.749 
p-value : 0.000
--> 이탈고객의 총 거래 금액 평균과 기존고객 총 거래 금액 평균의 차이는 0이 아니므로 귀무가설을 기각한다.


## 6. 총 거래 건수
* 귀무가설 : 이탈고객과 기존고객 사이의 총 거래 건수 평균의 차이가 없다.
-> 이탈고객의 총 거래 금액 평균과 기존고객 총 거래 건수 평균의 차이는 0이다.
-> Existing_mean - Attrited_mean = 0
* 대립가설 : 이탈고객과 기존고객 사이의 총 거래 건수 평균의 차이가 있다.
-> 이탈고객의 총 거래 금액 평균과 기존고객 총 거래 건수 평균의 차이는 0이 아니다.
-> Existing_mean - Attrited_mean != 0

In [61]:
from scipy import stats
Total_Trans_Ct= df.Total_Trans_Ct    

#독립표본 t-검정
#변수생성
Existing = df[df.Attrition_Flag == "Existing Customer"] #Existing Customer
Attrited = df[df.Attrition_Flag == "Attrited Customer"] #Attrited Customer

#Levene의 등분산 검정 
lresult = stats.levene(Existing.Total_Trans_Ct, Attrited.Total_Trans_Ct)
print('LeveneResult(F) : %.3f \np-value : %.3f' % (lresult))
if lresult.pvalue >= 0.05:
    bool_ = True
    print("-> 총 거래 건수 평균의 분산은 이분산이다.")
else:
    bool_ = False
    print('-> 총 거래 건수 평균의 분산은 등분산이다.')

result = stats.ttest_ind(Existing.Total_Trans_Ct, Attrited.Total_Trans_Ct, equal_var=bool_) 
print('t statistic : %.3f \np-value : %.3f' % (result))
if result.pvalue >= 0.05:
    print("--> 이탈고객의 총 거래 건수 평균과 기존고객 총 거래 건수 평균의 차이는 0이므로 귀무가설을 채택한다.")
else:
    print('--> 이탈고객의 총 거래 건수 평균과 기존고객 총 거래 건수 평균의 차이는 0이 아니므로 귀무가설을 기각한다.')

LeveneResult(F) : 199.415 
p-value : 0.000
-> 총 거래 건수 평균의 분산은 등분산이다.
t statistic : 27.403 
p-value : 0.000
--> 이탈고객의 총 거래 건수 평균과 기존고객 총 거래 건수 평균의 차이는 0이 아니므로 귀무가설을 기각한다.


## 7. 거래 수의 변화
* 귀무가설 : 이탈고객과 기존고객 사이의 거래 수의 변화 평균의 차이가 없다.
-> 이탈고객의 총 거래 금액 평균과 기존고객 거래 수의 변화 평균의 차이는 0이다.
-> Existing_mean - Attrited_mean = 0
* 대립가설 : 이탈고객과 기존고객 사이의 거래 수의 변화 평균의 차이가 있다.
-> 이탈고객의 총 거래 금액 평균과 기존고객 거래 수의 변화 평균의 차이는 0이 아니다.
-> Existing_mean - Attrited_mean != 0

In [62]:
from scipy import stats
Total_Ct_Chng_Q4_Q1= df.Total_Ct_Chng_Q4_Q1    

#독립표본 t-검정
#변수생성
Existing = df[df.Attrition_Flag == "Existing Customer"] #Existing Customer
Attrited = df[df.Attrition_Flag == "Attrited Customer"] #Attrited Customer

#Levene의 등분산 검정 
lresult = stats.levene(Existing.Total_Ct_Chng_Q4_Q1, Attrited.Total_Ct_Chng_Q4_Q1)
print('LeveneResult(F) : %.3f \np-value : %.3f' % (lresult))
if lresult.pvalue >= 0.05:
    bool_ = True
    print("-> 거래 수의 변화 평균의 분산은 이분산이다.")
else:
    bool_ = False
    print('-> 거래 수의 변화 평균의 분산은 등분산이다.')

result = stats.ttest_ind(Existing.Total_Ct_Chng_Q4_Q1, Attrited.Total_Ct_Chng_Q4_Q1, equal_var=bool_) 
print('t statistic : %.3f \np-value : %.3f' % (result))
if result.pvalue >= 0.05:
    print("--> 이탈고객의 거래 수의 변화 평균과 기존고객 거래 수의 변화 평균의 차이는 0이므로 귀무가설을 채택한다.")
else:
    print('--> 이탈고객의 거래 수의 변화 평균과 기존고객 거래 수의 변화 평균의 차이는 0이 아니므로 귀무가설을 기각한다.')

LeveneResult(F) : 14.603 
p-value : 0.000
-> 거래 수의 변화 평균의 분산은 등분산이다.
t statistic : 20.003 
p-value : 0.000
--> 이탈고객의 거래 수의 변화 평균과 기존고객 거래 수의 변화 평균의 차이는 0이 아니므로 귀무가설을 기각한다.


## 8. 평균 카드 이용률
* 귀무가설 : 이탈고객과 기존고객 사이의 평균 카드 이용률 평균의 차이가 없다.
-> 이탈고객의 총 거래 금액 평균과 기존고객 평균 카드 이용률 평균의 차이는 0이다.
-> Existing_mean - Attrited_mean = 0
* 대립가설 : 이탈고객과 기존고객 사이의 평균 카드 이용률 평균의 차이가 있다.
-> 이탈고객의 총 거래 금액 평균과 기존고객 평균 카드 이용률 평균의 차이는 0이 아니다.
-> Existing_mean - Attrited_mean != 0

In [63]:
from scipy import stats
Avg_Utilization_Ratio = df.Avg_Utilization_Ratio     

#독립표본 t-검정
#변수생성
Existing = df[df.Attrition_Flag == "Existing Customer"] #Existing Customer
Attrited = df[df.Attrition_Flag == "Attrited Customer"] #Attrited Customer

#Levene의 등분산 검정 
lresult = stats.levene(Existing.Avg_Utilization_Ratio , Attrited.Avg_Utilization_Ratio )
print('LeveneResult(F) : %.3f \np-value : %.3f' % (lresult))
if lresult.pvalue >= 0.05:
    bool_ = True
    print("-> 평균 카드 이용률 평균의 분산은 이분산이다.")
else:
    bool_ = False
    print('-> 평균 카드 이용률 평균의 분산은 등분산이다.')

result = stats.ttest_ind(Existing.Avg_Utilization_Ratio, Attrited.Avg_Utilization_Ratio , equal_var=bool_) 
print('t statistic : %.3f \np-value : %.3f' % (result))
if result.pvalue >= 0.05:
    print("--> 이탈고객의 평균 카드 이용률 평균과 기존고객 평균 카드 이용률 평균의 차이는 0이므로 귀무가설을 채택한다.")
else:
    print('--> 이탈고객의 평균 카드 이용률 평균과 기존고객 평균 카드 이용률 평균의 차이는 0이 아니므로 귀무가설을 기각한다.')

LeveneResult(F) : 60.080 
p-value : 0.000
-> 평균 카드 이용률 평균의 분산은 등분산이다.
t statistic : 12.683 
p-value : 0.000
--> 이탈고객의 평균 카드 이용률 평균과 기존고객 평균 카드 이용률 평균의 차이는 0이 아니므로 귀무가설을 기각한다.


## 9. 고객 나이
* 귀무가설 : 이탈고객과 기존고객 사이의 고객 나이 평균의 차이가 없다.
-> 이탈고객의 총 거래 금액 평균과 기존고객 고객 나이 평균의 차이는 0이다.
-> Existing_mean - Attrited_mean = 0
* 대립가설 : 이탈고객과 기존고객 사이의 고객 나이 평균의 차이가 있다.
-> 이탈고객의 총 거래 금액 평균과 기존고객 고객 나이 평균의 차이는 0이 아니다.
-> Existing_mean - Attrited_mean != 0

In [64]:
from scipy import stats
Customer_Age  = df.Customer_Age      

#독립표본 t-검정
#변수생성
Existing = df[df.Attrition_Flag == "Existing Customer"] #Existing Customer
Attrited = df[df.Attrition_Flag == "Attrited Customer"] #Attrited Customer

#Levene의 등분산 검정 
lresult = stats.levene(Existing.Customer_Age, Attrited.Customer_Age  )
print('LeveneResult(F) : %.3f \np-value : %.3f' % (lresult))
if lresult.pvalue >= 0.05:
    bool_ = True
    print("-> 고객 나이 평균의 분산은 이분산이다.")
else:
    bool_ = False
    print('-> 고객 나이 평균의 분산은 등분산이다.')

result = stats.ttest_ind(Existing.Customer_Age, Attrited.Customer_Age, equal_var=bool_) 
print('t statistic : %.3f \np-value : %.3f' % (result))
if result.pvalue >= 0.05:
    print("--> 이탈고객의 고객 나이 평균과 기존고객 고객 나이 평균의 차이는 0이므로 귀무가설을 채택한다.")
else:
    print('--> 이탈고객의 고객 나이 평균과 기존고객 고객 나이 평균의 차이는 0이 아니므로 귀무가설을 기각한다.')

LeveneResult(F) : 1.496 
p-value : 0.221
-> 고객 나이 평균의 분산은 이분산이다.
t statistic : -0.411 
p-value : 0.681
--> 이탈고객의 고객 나이 평균과 기존고객 고객 나이 평균의 차이는 0이므로 귀무가설을 채택한다.
